In [2]:
from collections import Counter
from datetime import datetime
from pandas import Series

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re

### Extracting papers between 2016 and 2020 ###

In [3]:
### extracting bibliography data between 2016 and 2020 in Papers.txt which is given by MAG ###
## extracting online/offline data by offline date and online date
## index number 8(offline date) and 9(online date)
## So It is possible to include duplicated papers because it is divided by online and offline date

filename = "Papers.txt" #MAG data
chunksize = 10000

for cnt, chunk in enumerate(pd.read_table(filename, chunksize=chunksize, header=None, quoting = 3)):
    i = 0
    number = 0
    try:
        while i == number:
            cnt == number
            chunk[8]= pd.to_datetime(chunk[8], format='%Y-%m-%d') #chunk[8] -> Date, chunk[9] -> OnlineDate
            df = chunk.loc[chunk[8]> '2015', :]
            df.to_csv('papers_2016.txt', index=False, header=None, sep="\t", mode='a')
            i +=1
    except:
        pass
    
for cnt, chunk in enumerate(pd.read_table(filename, chunksize=chunksize, header=None, quoting = 3)):
    i = 0
    number = 0
    try:
        while i == number:
            cnt == number
            chunk[9]= pd.to_datetime(chunk[9], format='%Y-%m-%d') #chunk[8] -> Date, chunk[9] -> OnlineDate
            df = chunk.loc[chunk[9]> '2015', :]
            df.to_csv('papers_2016_online.txt', index=False, header=None, sep="\t", mode='a')
            i +=1
    except:
        pass
    
filename = "papers_2016.txt"
chunksize = 10000

for cnt, chunk in enumerate(pd.read_table(filename, chunksize=chunksize, header=None, quoting = 3)):
    i = 0
    number = 0
    while i == number:
        cnt == number
        chunk = chunk[[0, 3, 7, 8, 9, 11, 12]]
        chunk[13] = 1
        chunk.to_csv('papers_2016_sub1.txt', index=False, header=None, sep="\t", mode='a')
        i +=1
        
filename = "papers_2016_online.txt"
chunksize = 10000

for cnt, chunk in enumerate(pd.read_table(filename, chunksize=chunksize, header=None, quoting = 3)):
    i = 0
    number = 0
    while i == number:
        cnt == number
        chunk = chunk[[0, 3, 7, 8, 9, 11, 12]]
        chunk[13] = 2
        chunk.to_csv('papers_2016_sub2.txt', index=False, header=None, sep="\t", mode='a')
        i +=1
        
df1 = pd.read_table("papers_2016_sub1.txt", header=None)
df2 = pd.read_table("papers_2016_sub2.txt", header=None)
df3 = pd.concat([df1, df2]) 
df3 = df3.drop_duplicates()

df3 = df3.loc[df3[1].notnull(), :] # paper_type is not null
#papers_2016_sub_total: paperid, paper_type, year, date, onlinedate, journalid, conferid
df3.to_csv("papers_2016_sub_total.txt", index=False, header=None, sep="\t")

### Spliting date by year and month ###

In [3]:
### Pretreatment date ###
## Data has two date(offline and online)
## Formation of date is "2020-01-01", so let's split date by using "-"

df = pd.read_table("papers_2016_sub_total.txt", header=None)

year1 = []; year2 = []

for i in df[3]:
    i = str(i)
    j = i.split("-")[0]
    j = re.sub('[^0-9]', '', j) #some data dosent'have date format
    year1.append(j)

for i in df[4]:
    i = str(i)
    j = i.split("-")[0]
    j = re.sub('[^0-9]', '', j) 
    year2.append(j)

df["year1"] = year1    
df["year2"] = year2

df.loc[df['year1']=='', 'year1'] = 0
df.loc[df['year2']=='', 'year2'] = 0

df['year1'] = df['year1'].astype(int)
df['year2'] = df['year2'].astype(int)

over1 = df.loc[(df['year1']>=2016)&(df['year1']<=2020), :]
over1 = over1.loc[(over1['year2']==0)|(over1['year2']>=2016)&(over1['year2']<=2020), :]

over2 = df.loc[(df['year1']>=2016)&(df['year2']>=2016)&(df['year2']<=2020), :]
over2 = over2.loc[(over2['year1']==0)|(over2['year1']>=2016)&(over2['year1']<=2020), :]

off = over1.loc[over1[7]==1, :] #date -> 1, onlinedate ->2
on = over2.loc[over2[7]==2, :]

df2 = pd.concat([off, on])

month1 = []; month2 = []
for i in df2[3]:
    i = str(i)
    if "-" in i:
        j = i.split("-")[1]
        j = re.sub('[^0-9]', '', j)
    else:
        j = 0
    month1.append(j)
    
for i in df2[4]:
    i = str(i)
    if "-" in i:        
        j = i.split("-")[1]
        j = re.sub('[^0-9]', '', j)
    else:
        j = 0
    month2.append(j)
    
df2['month1'] = month1
df2['month2'] = month2

df2['month1'] = df2['month1'].astype(int)
df2['month2'] = df2['month2'].astype(int)

df2 = df2[[0, 1, 5, 6, 7, 'year1', 'year2', 'month1', 'month2']]
df2.columns = ['paperid', 'paper_type', 'jourid', 'confid', 'onoff', 'year1', 'year2', 'month1', 'month2']

df2.loc[(df2['month1']>=1)&(df2['month1']<=3), 'quarter1']=1
df2.loc[(df2['month1']>=4)&(df2['month1']<=6), 'quarter1']=2
df2.loc[(df2['month1']>=7)&(df2['month1']<=9), 'quarter1']=3
df2.loc[(df2['month1']>=10)&(df2['month1']<=12), 'quarter1']=4

df2.loc[(df2['month2']>=1)&(df2['month2']<=3), 'quarter2']=1
df2.loc[(df2['month2']>=4)&(df2['month2']<=6), 'quarter2']=2
df2.loc[(df2['month2']>=7)&(df2['month2']<=9), 'quarter2']=3
df2.loc[(df2['month2']>=10)&(df2['month2']<=12), 'quarter2']=4

#paeprs_2016_sub_total2:'paperid', 'paper_type', 'jourid', 'confid', 'onoff', 'year1', 'year2', 'month1', 'month2', 'quarter1', 'quarter2'
df2.to_csv('papers_2016_sub_total2.txt', index=False, sep="\t")

# we use data between 2016 and 2020
df2["paperid"].to_csv('paperid.txt', header=None, index=False, sep="\t")

### Classifying gender and number of authors ###

In [14]:
### extract authorid and author sequence number between 2016 and 2020 ###

filename = "PaperAuthorAffiliations.txt" #MAG data
chunksize = 10000

for cnt, chunk in enumerate(pd.read_table(filename, chunksize=chunksize, header=None, quoting = 3)):
    i = 0
    number = 0
    while i == number:
        cnt == number
        df =chunk[[0, 1, 3]] #paperid, authorid, authorsequencenumber
        df.to_csv('paperauthor.txt', index=False, header=None, sep="\t", mode='a')
        i +=1
        
paper_id = pd.read_table('paperid.txt', header=None) #paperid between 2016 and 2020
paper_id = paper_id.drop_duplicates()
paper_id["over16"] = 1
paper_id.rename(columns = {0: 'id'}, inplace = True)

#filtering paperid in paperauthor between 2016 and 2020
df = pd.read_table("paperauthor", header=None, encoding = 'ISO-8859-1', quoting = 3) 
df.rename(columns = {0: 'id'}, inplace = True)

df = pd.merge(df, paper_id, left_on='id', right_on='id', how='left')
df = df.loc[df['over16']==1, :]

#paper_author: paperid, authorid, authorsequencenumber, over16
df.to_csv("paper_author.txt", index=False, header=None, sep="\t", mode='a') 

#extracting authorid between 2016 and 2020
filename = "paper_author.txt"

for cnt, chunk in enumerate(pd.read_table(filename, chunksize=chunksize, header=None, quoting = 3)):
    i = 0
    number = 0
    while i == number:
        cnt == number
        df =chunk[[1]]
        df.to_csv('authorid.txt', index=False, header=None, sep="\t", mode='a')
        i +=1

In [ ]:
### classifying gender by using author's name ###
## extracting authorid paperid authorname gender

author = pd.read_table('authorid.txt', header=None, error_bad_lines=False) #load authorid between 2016 and 2020
author = author.drop_duplicates()
author.rename(columns = {0: 'id'}, inplace = True)
author["over16"] = 1

filename = "Authors.txt" #MAG data

for cnt, chunk in enumerate(pd.read_table(filename, chunksize=chunksize, header=None, quoting = 3)):
    i = 0
    number = 0
    while i == number:
        cnt == number
        df =chunk[[0, 2, 4]] #authorid, nomalized name, lastaffiliationid
        df.to_csv('authors_sub.txt', index=False, header=None, sep="\t", mode='a')
        i +=1
        
df = pd.read_table('authors_sub', header=None, encoding = 'ISO-8859-1', quoting = 3) #, 
df.rename(columns = {0: 'id'}, inplace = True)

df = pd.merge(df, author, left_on='id', right_on='id', how='left')
df = df.loc[df['over16']==1, :]

#author_name: authorid, nomalized name, lastaffiliationid, over16
df.to_csv("author_name.txt", index=False, header=None, sep="\t")

#We got gender information of "author_name.txt"'s nomalized name by using genderize api
#We then infer gender from the authors' first name, which is the first token of the full name split by space using genderize
gender = pd.read_table('gender_infor.txt') #Gnderize data(https://genderize.io/)

author_name = pd.read_table('author_name.txt', header=None)
author_name['name'] = author_name.apply(lambda x : x[1].split(" ")[0], axis = 1)
author_name = pd.merge(author_name, gender, left_on= 'name', right_on='name', how='left')

author_name.rename(columns = {0: 'authorid'}, inplace = True)
author_name = author_name[['authorid', 'gender']]
author_name.loc[author_name['gender']=='None', 'gender'] = np.nan 
author_name = author_name.loc[author_name['gender'].notnull(), :]

female = {'female':1, 'male':0}
author_name = author_name.replace(female)

#author_gender: authorid, gender(female=1, male=0)
author_name.to_csv('author_gender.txt', sep="\t", index=False) 

In [17]:
### making variable(female_have, female_first, female_last) and number of authors in each paper
#female_have: including any female author in paper=1, male=0, unknown=np.nan 
#female_first first author is female=1, male=0, unknown=np.nan
#female_last: last author is female=1, male=0, 알수없음=np.nan

author_gender = pd.read_table('author_gender.txt')
author_gender.columns = ['authorid', 'gender']

df = pd.read_table("paper_author.txt", header=None)
df = df.drop([3], axis=1)
df.columns = ['paperid', 'authorid', 'sequence_number']
df = pd.merge(df, author_gender, on='authorid', how='left')

## female_have
female1 = df[['paperid', 'gender']]
female1 = female1.drop_duplicates()

lst1 = list(female1.loc[female1['gender']==1, :]['paperid']) #female
lst2 = list(female1.loc[female1['gender']==0, :]['paperid'])
lst3 = list(set(lst2) - set(lst1)) #male

female = pd.DataFrame(lst1)
female['female_have'] =1

male = pd.DataFrame(lst3)
male['female_have'] = 0

lst = pd.concat([female, male])

female_have = pd.DataFrame(list(set(female1['paperid'])))
female_have = pd.merge(female_have, lst, on=0, how='left')
female_have.rename(columns = {0: 'paperid'}, inplace = True)

## female_first
female_first = df.loc[df['sequence_number']==1, :]
female_first = female_first.drop_duplicates()
female_first = female_first[['paperid', 'gender']]
female_first.rename(columns = {'gender': 'female_first'}, inplace = True)

## female_last
female_last = df.loc[df['sequence_number']!=1, :]
female_last = female_last.sort_values(by=['paperid', 'sequence_number'] ,ascending=True)
female_last = female_last.drop_duplicates(['paperid'], keep='last')
female_last = female_last[['paperid', 'gender']]
female_last.rename(columns = {'gender': 'female_last'}, inplace = True)

## number of authors
author_n = df[['paperid', 'sequence_number']]
author_n = author_n.sort_values(by=['paperid', 'sequence_number'] ,ascending=True)
author_n = author_n.drop_duplicates(['paperid'], keep='last')

author_n.rename(columns = {'sequence_number': 'author_n'}, inplace = True)


female = pd.merge(female_have, female_first, on='paperid', how='left')
female = pd.merge(female, female_last, on='paperid', how='left')
female = pd.merge(female, author_n, on='paperid', how='left')

female.to_csv("female.txt", index=False, sep="\t")

### Fields of papers ###

In [2]:
### Making variable which is fields of papers ###

## extracing data in "PaperFieldsOfStudy.txt" between 2016 and 2020
paper_id = pd.read_table('paperid.txt', header=None) #load paperid between 2016 and 2020
paper_id = paper_id.drop_duplicates()
paper_id["over16"] = 1
paper_id.rename(columns = {0: 'id'}, inplace = True)

#PapersFieldsOfStudy: paperid fieldofstudyid over16
df = pd.read_table("PaperFieldsOfStudy.txt", header=None, encoding = 'ISO-8859-1', quoting = 3) #MAG data
df.rename(columns = {0: 'id'}, inplace = True)

df = pd.merge(df, paper_id, left_on='id', right_on='id', how='left')
df = df.loc[df['over16']==1, :]
df.to_csv("paper_field.txt", index=False, header=None, sep="\t", mode='a')

##Using fields list of MAG
#In PaperFieldsOfStudy.txt, each paper has multiple level of fields
#We then use the most specific fields(level0)
#Each paper has also multiple field in level0 and socre means corresponding degree in each area
#We use a field when the score is the highest.
journal = pd.read_table('FieldsOfStudy.txt', header=None) #MAG data
journal_id = pd.DataFrame(journal.loc[journal[5]==0, :][0])
journal_id = journal_id.reset_index(drop= True, inplace=False)
journal_id.rename(columns = {0: 'field_id'}, inplace = True)
journal_id['level0'] = 1

df = pd.read_table("paper_field.txt", header=None, encoding = 'ISO-8859-1', error_bad_lines=False)
df.rename(columns = {1: 'field_id'}, inplace = True)
df = df.loc[df[3].notnull(), :]
    
df['field_id'] = df['field_id'].astype(float)
df['field_id'] = df['field_id'].astype('int64')
    
df = pd.merge(df, journal_id, left_on='field_id', right_on='field_id', how='left')
df = df.loc[df['level0']==1, :]

#paper_jour: paperid, fieldofstudyid, score
df.to_csv("paper_jour.txt", index=False, header=None, sep="\t") 

paper_jour= pd.read_table("paper_jour.txt", sep="\t", header=None)
paper_jour= paper_jour.sort_values(by=[0, 2] ,ascending=False)

paper_jour2 = paper_jour.groupby(0)[2].rank(method="min", ascending=False)
paper_jour2 = pd.DataFrame(paper_jour2)
paper_jour["rank"] = paper_jour2[2]

paper_jour.columns = ['paperid', 'field_id', 'score', 'over16', 'level0', 'rank']

## field_n, number of fields
field_n = paper_jour.drop_duplicates(['paperid'], keep='last')
field_n = field_n[['paperid', 'rank']]
field_n.rename(columns = {'rank':"field_n"}, inplace = True)

## Making variable which the three high-scoring fields
## field1, field2, field3
paper_jour = paper_jour[['paperid', 'field_id', 'rank']]

rank1 = paper_jour.loc[paper_jour["rank"]==1, :]
rank2 = paper_jour.loc[paper_jour["rank"]==2, :]
rank3 = paper_jour.loc[paper_jour["rank"]==3, :]

rank1.rename(columns = {'field_id': 'field1'}, inplace = True)
rank2.rename(columns = {'field_id': 'field2'}, inplace = True)
rank3.rename(columns = {'field_id': 'field3'}, inplace = True)

rank = pd.merge(rank1, rank2, on='paperid', how='left')
rank = pd.merge(rank, rank3, on='paperid', how='left')
rank = rank[['paperid', 'field1', 'field2', 'field3']]

#rank: paperid, field1, field2, field3
rank = pd.merge(rank, field_n, on='paperid', how='left')

## narrowing down fields from 19 to 5
tag = pd.read_table('discipline.txt',sep='\t', header=None) #we reclassify fields into 5 and make discipline.txt
tag = tag.reset_index(drop=True).reset_index()
tag[3] = tag.apply(lambda x : x['index']+11, axis = 1)
tag.columns = ['index', 'field_id', 'field', 'id2', 'id1']

dic = {'humanities':1, 'engineering':2, 'socialscience':3, 'nature':4, 'medicine':5}
tag = tag.replace(dic)

tag1 = dict(zip(tag.field_id, tag.id1))
tag2 = dict(zip(tag.field_id, tag.id2))

rank['field_id1'] = rank['field1']
rank['field_id1'] = rank['field_id1'].replace(tag1)

rank['field_id2'] = rank['field1']
rank['field_id2'] = rank['field_id2'].replace(tag2)

rank = rank[["paperid", "field_id1", "field_id2", "field_n", "field1", "field2", "field3"]]
rank.to_csv("rank.txt", index=False, sep="\t")

### Countries of papers ###

In [2]:
### Figure out countries of authors using affiliation's information ###

paper_author = pd.read_table("paper_author.txt", header=None)
paper_author = paper_author.drop([3], axis=1)
paper_author.columns = ['paperid', 'authorid', 'sequence_number']

author_name = pd.read_table('author_name.txt', header=None)
author_name = author_name.drop([1, 3], axis=1)
author_name.columns = ['authorid', 'affiliation']

paper_author = pd.merge(paper_author, author_name, on='authorid', how='left')

## Affiliaitons.txt give a information of author's affiliation
paper_aff = pd.read_table('Affiliations.txt', header=None) #MAG data
paper_aff = paper_aff[[0, 10]]
paper_aff.rename(columns = {0: 'affiliation', 10:'country'}, inplace = True)

paper_author = pd.merge(paper_author, paper_aff, on='affiliation', how='left')

paper_author.to_csv("pa_au_aff.txt", index=False, sep='\t')

In [4]:
### country by each author and by sequence number
## country, country1, country2, country3

pa_au_aff = pd.read_table("pa_au_aff.txt")

country = pd.DataFrame(list(set(pa_au_aff['paperid'])))
country.rename(columns = {0: 'paperid'}, inplace = True)

pa_au_aff = pa_au_aff.drop_duplicates()

country1 = pa_au_aff.loc[pa_au_aff['sequence_number']==1, :]
country1 = country1[['paperid', 'country']]

country2 = pa_au_aff.loc[pa_au_aff['sequence_number']==2, :]
country2 = country2[['paperid', 'country']]

country3 = pa_au_aff.loc[pa_au_aff['sequence_number']==3, :]
country3 = country3[['paperid', 'country']]

country = pd.merge(country, country1, on='paperid', how='left')
country = pd.merge(country, country2, on='paperid', how='left')
country = pd.merge(country, country3, on='paperid', how='left')

country.columns = ['paperid', 'country1', 'country2', 'country3']

### number of countries ###
##country_n
country_n = pa_au_aff.loc[pa_au_aff['country'].notnull(), :]
country_n = country_n[['paperid', 'country']]
country_n = country_n.drop_duplicates()

country_n = country_n.groupby(['paperid']).count().reset_index()
country_n.rename(columns = {'country': 'country_n'}, inplace = True)

country = pd.merge(country, country_n, on='paperid', how='left')

country.to_csv("country.txt", index=False, sep='\t')

### Making individual/national characteristics ###

In [ ]:
### individual characteristics

### h-index by author and affiliation 
pa_au_aff = pd.read_table("pa_au_aff.txt")
pa_au_aff = pa_au_aff[['paperid', 'authorid', 'affiliation']]

aff = pd.read_table("MAG_210215_AffiliationInfo10Y.tsv", sep="\t") #MAG metadata
aff = aff[['AffiliationId', 'HIdx(JourConfRepo)']]
aff.rename(columns = {'HIdx(JourConfRepo)': 'aff_hidx', 'AffiliationId':'affiliation'}, inplace = True)

aut = pd.read_table("MAG_210215_AuthorInfo10Y.tsv", sep="\t")
aut = aut[['AuthorId', 'HIdx(JourConfRepo)']]
aut.rename(columns = {'HIdx(JourConfRepo)': 'aut_hidx', 'AuthorId':'authorid'}, inplace = True)

pa_au_aff = pd.merge(pa_au_aff, aff, on='affiliation', how='left')
pa_au_aff = pd.merge(pa_au_aff, aut, on='authorid', how='left')

pa_au_aff = pa_au_aff.loc[pa_au_aff['authorid'].notnull(), :]

### academic_age
aca_age = pd.read_table("MAG_210215_AuthorInfo.tsv") #MAG metadata

aca_age = aca_age[['AuthorId', 'FirstPub(DateJourConfRepo)']]
aca_age = aca_age.loc[aca_age['FirstPub(DateJourConfRepo)'].notnull(), :]

#deleting outlier
aca_age = aca_age.loc[(aca_age['FirstPub(DateJourConfRepo)']>=1960)&(aca_age['FirstPub(DateJourConfRepo)']<=2020), :]

#calculating academic age from present to time of a first paper
aca_age['aca_age']= aca_age.apply(lambda x : 2020 - x['FirstPub(DateJourConfRepo)'], axis = 1)

aca_age = aca_age.drop(['FirstPub(DateJourConfRepo)'], axis=1) #AuthorId, aca_age
aca_age.rename(columns = {'AuthorId': 'authorid'}, inplace = True)

aca_age.to_csv("aca_age1.txt", index=False, sep='\t')

aca_age = pd.read_table("aca_age1.txt")
pa_au_aff = pd.merge(pa_au_aff, aca_age, on='authorid', how='left')
pa_au_aff.to_csv("pa_au_aff2.txt", index=False, sep='\t')

In [20]:
### national characteristics ###

## we merge between mag and data from othersources by country 

##using iso_code in MAG data
paper_aff = pd.read_table('Affiliations.txt', header=None) #MAG data
paper_aff = paper_aff[[0, 10]]
paper_aff.rename(columns = {0: 'AffiliationId', 10:'iso_code_3'}, inplace = True)
paper_aff = paper_aff.loc[paper_aff['iso_code_3'].notnull(), :]

#total cases per million/total deaths per million
covid = pd.read_csv('covid.csv') #https://github.com/owid/covid-19-data/tree/master/public/data
covid['date'] = pd.to_datetime(covid['date'])
covid = covid.loc[covid['date']<"2021"] #last date in 2020
covid = covid.sort_values(by=['iso_code', 'date'] ,ascending=True)
covid = covid.drop_duplicates(['iso_code'], keep='last')

covid = covid.loc[covid['total_cases_per_million'].notnull(), :]
covid = covid.loc[covid['total_deaths_per_million'].notnull(), :]

covid = covid[['iso_code', 'iso_code_3', 'total_cases_per_million', 'total_deaths_per_million',]]
covid = covid.loc[covid['iso_code_3'].notnull(), :]

#Workplace/Residential
gm = pd.read_csv("Global_Mobility_Report.csv", encoding='utf-8') #https://www.google.com/covid19/mobility

gm['date'] = pd.to_datetime(gm['date'])
gm = gm.loc[gm['date']<"2021"] #2020년 마지막 보고 기준
gm = gm.groupby(['country_region_code']).mean().reset_index()

gm.rename(columns = {'workplaces_percent_change_from_baseline': 'Workplace', 
                     'residential_percent_change_from_baseline':'Residential', 
                     'country_region_code': 'iso_code_3'}, inplace = True)
gm = pd.merge(gm, covid, on='iso_code_3', how='left')

gm = gm[['iso_code_3', 'Workplace', 'Residential']]

#gender equality
ggis = pd.read_csv('data_indicators_country.csv', encoding='cp949') #https://github.com/fverkroost/epjds-professional-gender-gaps
ggis = ggis[['iso3code', 'sex_ratio_linkedin_overall_filtered']]
#filter: The observation filter identifies those countries for which audience counts in that category of at least 1000 were available.

ggis.rename(columns = {'iso3code':'iso_code', 'sex_ratio_linkedin_overall_filtered': 'ggis1', 'sex_ratio_ilostat_prof_tech_filtered':'ggis2' }, inplace = True)

ggis1 = ggis.loc[ggis['ggis1'].notnull(), :][['iso_code', 'ggis1']]

#merging
paper_aff = pd.merge(paper_aff, covid, on='iso_code_3', how='left')
paper_aff = pd.merge(paper_aff, gm, on='iso_code_3', how='left')
paper_aff = pd.merge(paper_aff, ggis1, on='iso_code', how='left')

paper_aff.rename(columns = {'AffiliationId': 'affiliation'}, inplace = True)

paper_aff.to_csv("paper_aff.txt", index=False, sep='\t')

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [49]:
#merging individual and natioanl data
pa_au_aff = pd.read_table("pa_au_aff2.txt")

pa_au_aff = pd.merge(pa_au_aff, paper_aff, on='affiliation', how='left')

pa_au_aff = pa_au_aff.groupby(['paperid']).mean().reset_index()
pa_au_aff = pa_au_aff.drop(['authorid', 'affiliation'], axis=1)
 
pa_au_aff.to_csv("other_var.txt", index=False, sep='\t')

### Merging all variables and making dataset by paperid ###

In [37]:
### merging all variables and split online/offline data ###

date = pd.read_table('papers_2016_sub_total2.txt')

female = pd.read_table('female.txt')

rank = pd.read_table('rank.txt')

country = pd.read_table('country.txt')

other_var = pd.read_table('other_var.txt')

date = pd.merge(date, female, on='paperid', how='left')
date = pd.merge(date, rank, on='paperid', how='left')
date = pd.merge(date, country, on='paperid', how='left')
date = pd.merge(date, other_var, on='paperid', how='left')

#paper type
dic =  {"Repository": 1, "Journal": 2, "Thesis": 3, "Conference": 4, "Patent": 5,  "Book": 6, "BookChapter": 7, "Dataset": 8}

#saving offline data
offline = date.loc[date['onoff']==1, :]
offline['paper_type'] = offline['paper_type'].replace(dic)
offline = offline.loc[(offline['paper_type']<=2)|(offline['paper_type']==4), :]
offline.to_csv("offline_0627.txt", index=False, sep='\t')

#saving online data
online = date.loc[date['onoff']==2, :]
online['paper_type'] = online['paper_type'].replace(dic)
online = online.loc[(online['paper_type']<=2)|(online['paper_type']==4), :]
online.to_csv("online_0627.txt",index=False, sep='\t')

### Making dataset by authorid ###

In [ ]:
### making individual/national characteristics dataset by authorid ###
pa_au_aff = pd.read_table("pa_au_aff2.txt")

pa_au_aff = pd.merge(pa_au_aff, paper_aff, on='affiliation', how='left')

pa_au_aff = pa_au_aff.drop(['paperid'], axis=1)
pa_au_aff = pa_au_aff.drop_duplicates()

pa_au_aff = pa_au_aff.loc[pa_au_aff['affiliation'].notnull(), :]

pa_au_aff.to_csv("effect_var.txt", index=False, sep='\t')

In [ ]:
### aca_age_sub ###
pa_au_aff = pd.read_table("pa_au_aff.txt")
pa_au_aff = pa_au_aff[['paperid', 'authorid', 'sequence_number']]

aca_age = pd.read_table("aca_age1.txt")
pa_au_aff = pd.merge(pa_au_aff, aca_age, on='authorid', how='left')

pa_au_aff.to_csv("aca_age2.txt", index=False, sep='\t') #paperid, authorid, sequence_number, aca_age

In [ ]:
### making dataset by authorid - offline ###

##loading dataset(by paperid)
offline = pd.read_table('offline_0627.txt')

offline = offline.loc[offline['author_n']<= 50, :] #under 50
offline = offline.loc[offline['paper_type']==2, :] #only use journal
offline = offline.loc[offline['field_id1'].notnull(), :]

offline = offline[['paperid', 'paper_type', 'year1', 'month1', 'field_id1']]

aca_age2 = pd.read_table('aca_age2.txt')
aca_age2 = aca_age2.drop_duplicates()

author_gender = pd.read_table("author_gender.txt")

aca_age2 = pd.merge(aca_age2, offline, on='paperid', how='left')
aca_age2 = aca_age2.loc[aca_age2['paper_type'].notnull(), :]

aca_age2 = pd.merge(aca_age2, author_gender, on='authorid', how='left')

aca_age2["age_year"] = aca_age2.apply(lambda x : x['aca_age'] - (2020 - x['year1']), axis = 1)

aca_age2 = aca_age2.sort_values(by=['paperid', 'sequence_number'] ,ascending=True)
aca_age2["id"] = aca_age2.apply(lambda x : str(x['paperid']) +','+ str(x['sequence_number']), axis = 1)

female_last = aca_age2.drop_duplicates(['paperid'], keep='last')
female_last["author_role"] = 2
female_last = female_last.loc[female_last['sequence_number']!=1, :]
female_last = female_last[['id', 'author_role']]

aca_age2 = pd.merge(aca_age2, female_last, on='id', how='left')

aca_age2.loc[aca_age2['sequence_number']==1, "author_role"] ="제1"
aca_age2.loc[aca_age2['author_role']==2, "author_role"] ="교신"
aca_age2.loc[aca_age2['author_role'].isnull(), "author_role"] ="기타"

aca_age2 = aca_age2.drop(['id', 'paper_type', "aca_age"], axis =1)

aca_age2.to_csv("author_role_plot_0627.txt", index=False, sep='\t')

In [6]:
### making dataset by authorid - online ###

##loading dataset(by paperid)
online = pd.read_table('online_0627.txt')

online = online.loc[online['author_n']<= 50, :]
online = online.loc[online['paper_type']==1, :] #only use repository
online = online.loc[online['field_id1'].notnull(), :]

online = online[['paperid', 'paper_type', 'year2', 'month2', 'field_id1']]

aca_age2 = pd.read_table('aca_age2.txt')
aca_age2 = aca_age2.drop_duplicates()

author_gender = pd.read_table("author_gender.txt")

aca_age2 = pd.merge(aca_age2, online, on='paperid', how='left')
aca_age2 = aca_age2.loc[aca_age2['paper_type'].notnull(), :]

aca_age2 = pd.merge(aca_age2, author_gender, on='authorid', how='left')

aca_age2["age_year"] = aca_age2.apply(lambda x : x['aca_age'] - (2020 - x['year1']), axis = 1)

aca_age2 = aca_age2.sort_values(by=['paperid', 'sequence_number'] ,ascending=True)
aca_age2["id"] = aca_age2.apply(lambda x : str(x['paperid']) +','+ str(x['sequence_number']), axis = 1)

female_last = aca_age2.drop_duplicates(['paperid'], keep='last')
female_last["author_role"] = 2
female_last = female_last.loc[female_last['sequence_number']!=1, :]
female_last = female_last[['id', 'author_role']]

aca_age2 = pd.merge(aca_age2, female_last, on='id', how='left')

aca_age2.loc[aca_age2['sequence_number']==1, "author_role"] ="제1"
aca_age2.loc[aca_age2['author_role']==2, "author_role"] ="교신"
aca_age2.loc[aca_age2['author_role'].isnull(), "author_role"] ="기타"

aca_age2 = aca_age2.drop(['id', 'paper_type', "aca_age"], axis =1)

aca_age2.to_csv("author_role_plot_on_0627.txt", index=False, sep='\t')

### Making dataset for XGBoost ###

In [14]:
### offline
offline = pd.read_table('offline_0627.txt')

offline = offline.loc[offline['author_n']<= 50, :]
offline = offline.loc[offline['paper_type']==2, :]
offline = offline.loc[offline['field_id1'].notnull(), :]

offline.rename(columns = {'year1': 'year'}, inplace = True)

off = offline[['paperid', 'year', 'female_have', 'female_first', 'female_last']]

author_role = pd.read_table('author_role_plot_0627.txt')

mean= author_role[['paperid', 'authorid']]
mean = pd.merge(mean, df_var, on='authorid', how='left')
mean = mean[['paperid', 'aca_age', 'aut_hidx', 'aff_hidx', 'ggis1', 'total_cases_per_million', 'total_deaths_per_million', 'Workplace', 'Residential']]
mean = mean.groupby(['paperid']).mean().reset_index()
mean = mean.drop_duplicates()

first = author_role.loc[author_role['author_role'] == '제1', :]
first = first[['paperid', 'authorid']]
first = pd.merge(first, df_var, on='authorid', how='left')
first = first[['paperid', 'aca_age', 'aut_hidx', 'aff_hidx', 'ggis1', 'total_cases_per_million', 'total_deaths_per_million', 'Workplace', 'Residential']]
first = first.drop_duplicates()
first.rename(columns = {'aca_age':'aca_age_f', 'aut_hidx':'aut_hidx_f', 'aff_hidx':'aff_hidx_f', 'ggis1':'ggis1_f', 
                        'total_cases_per_million':'cases_f', 'total_deaths_per_million':'deaths_f', 'Workplace':'work_f', 'Residential':'res_f'}, inplace = True)

last = author_role.loc[author_role['author_role'] == '교신', :]
last = last[['paperid', 'authorid']]
last = pd.merge(last, df_var, on='authorid', how='left')
last = last[['paperid', 'aca_age', 'aut_hidx', 'aff_hidx', 'ggis1', 'total_cases_per_million', 'total_deaths_per_million', 'Workplace', 'Residential']]
last = last.drop_duplicates()
last.rename(columns = {'aca_age':'aca_age_l', 'aut_hidx':'aut_hidx_l', 'aff_hidx':'aff_hidx_l', 'ggis1':'ggis1_l',
                       'total_cases_per_million':'cases_l', 'total_deaths_per_million':'deaths_l', 'Workplace':'work_l', 'Residential':'res_l'}, inplace = True)

logis = pd.merge(mean, first, on='paperid', how='left')
logis = pd.merge(logis, last, on='paperid', how='left')
logis = pd.merge(off, logis, on='paperid', how='left')

logis.to_csv("logistic_0828.txt", sep='\t', index=False)

In [15]:
### offline
online = pd.read_table('online_0627.txt')

online = online.loc[online['author_n']<= 50, :]
online = online.loc[online['paper_type']==1, :]
online = online.loc[online['field_id1'].notnull(), :]

online.rename(columns = {'year2': 'year'}, inplace = True)

on = online[['paperid', 'year', 'female_have', 'female_first', 'female_last']]

author_role = pd.read_table('author_role_plot_on_0627.txt')

mean= author_role[['paperid', 'authorid']]
mean = pd.merge(mean, df_var, on='authorid', how='left')
mean = mean[['paperid', 'aca_age', 'aut_hidx', 'aff_hidx', 'ggis1', 'total_cases_per_million', 'total_deaths_per_million', 'Workplace', 'Residential']]
mean = mean.groupby(['paperid']).mean().reset_index()
mean = mean.drop_duplicates()

first = author_role.loc[author_role['author_role'] == '제1', :]
first = first[['paperid', 'authorid']]
first = pd.merge(first, df_var, on='authorid', how='left')
first = first[['paperid', 'aca_age', 'aut_hidx', 'aff_hidx', 'ggis1', 'total_cases_per_million', 'total_deaths_per_million', 'Workplace', 'Residential']]
first = first.drop_duplicates()
first.rename(columns = {'aca_age':'aca_age_f', 'aut_hidx':'aut_hidx_f', 'aff_hidx':'aff_hidx_f', 'ggis1':'ggis1_f', 
                        'total_cases_per_million':'cases_f', 'total_deaths_per_million':'deaths_f', 'Workplace':'work_f', 'Residential':'res_f'}, inplace = True)

last = author_role.loc[author_role['author_role'] == '교신', :]
last = last[['paperid', 'authorid']]
last = pd.merge(last, df_var, on='authorid', how='left')
last = last[['paperid', 'aca_age', 'aut_hidx', 'aff_hidx', 'ggis1', 'total_cases_per_million', 'total_deaths_per_million', 'Workplace', 'Residential']]
last = last.drop_duplicates()
last.rename(columns = {'aca_age':'aca_age_l', 'aut_hidx':'aut_hidx_l', 'aff_hidx':'aff_hidx_l', 'ggis1':'ggis1_l',
                       'total_cases_per_million':'cases_l', 'total_deaths_per_million':'deaths_l', 'Workplace':'work_l', 'Residential':'res_l'}, inplace = True)

logis = pd.merge(mean, first, on='paperid', how='left')
logis = pd.merge(logis, last, on='paperid', how='left')
logis = pd.merge(on, logis, on='paperid', how='left')

logis.to_csv("logistic_0828_on.txt", sep='\t', index=False)